In [1]:
import cobra
import pandas as pd
from custom_functions_scRBA import *

In [2]:
model = cobra.Model('scRBA_model')
model.solver = 'cplex'

Using license file /home/hvdinh16/Workspace/Softwares/gurobi910/linux64/gurobi.lic
Academic license - for non-commercial use only - expires 2022-08-08


#### Add species to model

In [3]:
with open('./model/RBA_species.txt') as f:
    species = f.read().split('\n')
for m in species:
    met = cobra.Metabolite(m)
    model.add_metabolites(met)

#### Add reactions to model

In [4]:
# Add reactions
df_rxns = pd.read_excel('./model/RBA_stoichiometry.xlsx')
df_rxns.index = df_rxns.id.to_list()

In [5]:
for r in df_rxns.index:
    x = metabolites_dict_from_reaction_equation_RBA(df_rxns.reaction[r])
    met_dict = dict()
    for k,v in x.items():
        if is_number(v):
            met_dict[k] = v
    
    rxn = cobra.Reaction(r)
    model.add_reaction(rxn)
    rxn.reaction = build_reaction_equation_from_metabolites_dict_RBA(met_dict, arrow='-->')
    rxn.bounds = (0,1e7)

#### Save

In [6]:
cobra.io.save_json_model(model, './model/RBA_constantSij.cobra.json')